In [ ]:
import pandas as pd

# Import Table
fondos_mutuos_dataframe = pd.read_csv('fondos-mutuos-table-1.csv')

# Import 

# print(type(fondos_mutuos_dataframe["Fondo Mutuo"]))

## Task 1: Limpiar Columna "Fondo Mutuo" de la tabla 
La columna de los nombres de los fondos contienen el nombre del gestor:
    - CREDICORP ACCIONES GLOBAL CREDIFONDO SAF

Hay que remover CREDIFONDO SAF del nombre del Fondo. 

Hay que luego iterar sobre todos los nombres en la columna, y a cada uno quitarle el nombre del gestor. 

### Implementacion 1

In [ ]:
# Obtain set de gestores 
gestores = set(fondos_mutuos_dataframe["Administradora"])


# Iterate over pandas series. 

dataframe_one = fondos_mutuos_dataframe.copy() 

for fondo in dataframe_one["Fondo Mutuo"]: # problem: variable fondo holds each value in the dataframe, but does not access the value of the dataframe itself
    for gestor in gestores:
        fondo = fondo.replace(gestor, "").strip()
    print(fondo)  # problem: fondo is changed, but not the value in the fondos_mutuos_dataframe[]

print(dataframe_one["Fondo Mutuo"])

# This does not work because the values in the DataFrame are themselves not updated

### Implementacion 2

Accessing & updating the value in the dataframe, by using location/position on each iteration. 

In [ ]:
dataframe_2 = fondos_mutuos_dataframe.copy()

for i in range(len(dataframe_2)):
    fondo = dataframe_2.loc[i, "Fondo Mutuo"]
    for gestor in gestores:
        fondo = fondo.replace(gestor, "").strip()
    dataframe_2.loc[i, "Fondo Mutuo"] = fondo   # actual values in the "Fondo Mutuo" column is updated 

print("Result")
print(dataframe_2)

### Implementacion 3
Using .apply method for more concise code

In [ ]:
dataframe_3 = fondos_mutuos_dataframe.copy()

def remove_gestor(fondo, gestores):
    for gestor in gestores:
        fondo = fondo.replace(gestor, "").strip()
    return fondo 

dataframe_3["Fondo Mutuo"] = dataframe_3["Fondo Mutuo"].apply(remove_gestor, gestores = gestores)

print("RESULT: ")
print(dataframe_3["Fondo Mutuo"])

## Task 2: Agregar años de existencial del fondo

In [ ]:
dataframe_años = fondos_mutuos_dataframe.copy()
from datetime import datetime 

dataframe_años["Fec. Inicio Operación"] = pd.to_datetime(dataframe_años["Fec. Inicio Operación"], format='mixed')

current_year = datetime.now().year 

dataframe_años["Años"] = current_year - dataframe_años["Fec. Inicio Operación"].dt.year

print(dataframe_años.head())

## Task 3: Update Excel Error to Null

In [ ]:
import pandas as pd
fondos_mutuos_excelerror = pd.read_csv("fondos-mutuos-table-1.csv")

print(fondos_mutuos_excelerror["Avg Return Arithmetic"][1])

for i in range(len(fondos_mutuos_excelerror)):
    fondos_mutuos_excelerror.loc[i,"Standard Dev"] = fondos_mutuos_excelerror.loc[i,"Standard Dev"].replace("#DIV/0!", "")
    fondos_mutuos_excelerror.loc[i, "Avg Return Arithmetic"] = fondos_mutuos_excelerror.loc[i,"Avg Return Arithmetic"].replace("#DIV/0!", "")

print(fondos_mutuos_excelerror["Avg Return Arithmetic"][1])



## Task 4: Convert Std. Dev & Avg. Mean from String --> Numbers 

In [ ]:
import pandas as pd

fondos_mutuos_dataframe = pd.read_csv("fondos-mutuos-table-1.csv")

# Values Before 
print(fondos_mutuos_dataframe["Standard Dev"][0])
print(type(fondos_mutuos_dataframe["Standard Dev"][0]))
print("")


# Use pandas.to_numeric to convert values in the series from strings to --> numpy floats  
fondos_mutuos_dataframe["Standard Dev"] = pd.to_numeric(fondos_mutuos_dataframe["Standard Dev"], errors = "coerce")
fondos_mutuos_dataframe["Avg Return Arithmetic"] = pd.to_numeric(fondos_mutuos_dataframe["Avg Return Arithmetic"] , errors = "coerce")

print(fondos_mutuos_dataframe["Standard Dev"][0])
print(type(fondos_mutuos_dataframe["Standard Dev"][0]))
print("")

print("Avg Arithmetic:")
print(fondos_mutuos_dataframe["Avg Return Arithmetic"][0])
print(type(fondos_mutuos_dataframe["Avg Return Arithmetic"][0]))



## Task 5: Agregar Cumulative Return (total + anualized) & CAGR

In [ ]:
dataframe_GeometricMean = fondos_mutuos_dataframe.copy()

#print(type(dataframe_GeometricMean["Rentabilidad 2024"][1]))

print(dataframe_GeometricMean["Rentabilidad 2023"][2])

for i in range(len(dataframe_GeometricMean)):
    print(type(dataframe_GeometricMean["Rentabilidad 2023"][i]))



In [ ]:
import json

file = open('fondos-mutuos-data-2.json') 

# load fie to a python list object
fondos_mutuos_array = json.load(file)  
print(type(fondos_mutuos_array))

# Get dictionary of the first fund
fund_one = fondos_mutuos_array[0]
print(type(fund_one))

# get list of keys
fund_keys = list(fund_one.keys()) # uses .keys method to get a list of the keys of the dictionary

# reverse the list of keys
fund_keys.reverse()


investment = 100 

# iterate over the keys, if its a rentability key, multiply investment x return value
for key in fund_keys:
    if "Rentabilidad" in key:
        investment *= 1 + fund_one[key]
print("cumulative return : ", investment)

# cumulative return as a % 
cumulative_return = (investment - 100) / 100
print("cumulative return pct : " , cumulative_return)

# Create new cumulative return data point
fund_one["Cumulative Return"] = cumulative_return

print(fund_one)




In [ ]:
# formula for the CAGR and cumulaltive return on a dictionary with anual return values 

def cagr_cumulative_return(fund, starting_investment=100): 

    asset_value = starting_investment

    has_return_data = False 

    fund_keys = list(fund.keys())
    fund_keys.reverse()

    fund_years = 0 

    for key in fund_keys: 
        if "Rentabilidad" in key and fund[key] is not None: 
            asset_value *= 1 + fund[key]
            if "2024" in key:
                fund_years += 0.5  
            else: 
                fund_years += 1
            has_return_data = True

    if has_return_data:
        cumulative_return = ((asset_value - starting_investment) / starting_investment)
        
        cumulative_return_anualized = cumulative_return / fund_years

        fund["Total Cumulative Return"] = cumulative_return

        fund["Cumulative Return Period"] = fund_years

        fund["Annualized Cumulative Return"] = cumulative_return_anualized

        cagr = (asset_value / starting_investment)**(1/fund_years) - 1
        fund["CAGR"] = cagr

    
    else: 
        fund["Total Cumulative Return"] = None

    return fund


# Testing: 
fondos_array_copy = fondos_mutuos_array.copy() # make copy of array
print(fondos_array_copy[9]["Fondo Mutuo"])  
print(fondos_array_copy[9]) # one fund's data before function 

print("")

# Apply function to fund data 
cagr_cumulative_return(fondos_array_copy[9], 15000)
print(fondos_array_copy[9])

## Task 5: Sharpe Ratio
- calculate fund's sharpe ratio
- sharpe ratio is a measure of risk adjusted return
- return per unit of risk

sharpe ratio is given by: 
- S = (portfolio return - risk free rate) / standard dev 

Sharpe Ratio for this Case:
- S = (CAGR - average 10-year t-bill of last 10 years) / fund's standard dev of annualized returns

In [ ]:
risk_free_rate = .023 # average 10 year t-bill rate of the last 10 years

import json

file = open("fondos-mutuos-data-3.json")

fondos_mutuos_list = json.load(file)

# Testing with one fund dictionary
print(fondos_mutuos_list[0])

sharpe = (fondos_mutuos_list[0]["CAGR"] - risk_free_rate) / fondos_mutuos_list[0]["Standard Dev"]
fondos_mutuos_list[0]["Sharpe"] = sharpe

print(fondos_mutuos_list[0])


# Iterating through entire list of funds and calculating sharpe for each 
for i in range(len(fondos_mutuos_list)):
    cagr = fondos_mutuos_list[i].get("CAGR")

    stdev = fondos_mutuos_list[i].get("Standard Dev")  # using get handles cases where the "Standard Dev" key is not in the dictionary, returning null


    if cagr is not None and stdev is not None and stdev != 0:
        sharpe = (cagr - risk_free_rate) / stdev
        fondos_mutuos_list[i]["Sharpe Ratio"] = sharpe

    else: 
        fondos_mutuos_list[i]["Sharpe Ratio"] = None 

print(fondos_mutuos_list)




In [ ]:
# Function to calculate sharpe ratio. 

import json

file = open("fondos-mutuos-data-3.json")

fondos_mutuos_list = json.load(file)


def sharpe_ratio(fund, risk_free_rate=0.02):

    cagr = fund.get("CAGR")

    stdev = fund.get("Standard Dev")

    if cagr is not None and stdev is not None and stdev != 0:
        sharpe_ratio = (cagr - risk_free_rate) / stdev 
        fund["Sharpe Ratio"] = sharpe_ratio

    else: 
        fund["Sharpe Ratio"] = None

    return fund 


for fund in fondos_mutuos_list:
    sharpe_ratio(fund)

print(fondos_mutuos_list[1])
print(fondos_mutuos_list[2])

    

## Task 6: Categorias de Fondos
Task: agregar columna de categorias generales de fondos

#### Tipos de Fondo:
- renta variable
- renta fija
    -  corto plazo soles y dolares: 
        - fondos de dueda (renta fija) que invierten en mercado monetario (depositos a plazo y certificados de deposito). 
        - horizonte de inversion de corto plazo. no pagas rescate anticipado. 
    - inst. deuda 
        - mediano plazo
        - corto plazo
        - duracion flexible
    - instrumento de deuda
- estructurado
    - busca obtener una renta previamente determinada 
- renta mixta
    - instrumentos de deuda de distintos plazos + acciones / fondos etc. 
- flexible
- fondo de fondos


### Implementacion 1:

In [ ]:
import pandas as pd

"""
This script is meant to add a new column called "Categoria" to the dataframe: fondos_mutuos_dataframe_3. 
    - The value in the "Categoria" column will depend on the value in the "Tipo Fondo" column
    - If the row's value for the "Tipo Fondo" column is in the categrias_renta_fija list, then add "Renta Fija" as its value for the "Categoria" Column
    - Else, if the row's value for the "Tipo Fondo" column is in the categorias_renta_mixta list, then add "Renta Mixta" as its value for the "Categoria" column
    - Else, if the row's value for the "Tipo Fondo" column is either "Flexible" or "Flexible Dolares" then add "Flexible" as its value for the "Categoria" column
    - Else: set the row's value for the "Categoria" column = the row's value in the "Tipo Fondo" column

"""
fondos_mutuos_dataframe_3 = pd.read_csv("fondos-mutuos-table-3.csv")

# Get set of unique type of funds from the "Tipo Fondo" column
tipo_de_fondos = set(fondos_mutuos_dataframe_3["Tipo Fondo"])

#for i in tipo_de_fondos:
#    print(i)

categorias_renta_fija = ["Corto Plazo Soles","Corto Plazo Dólares", "Inst. Deuda Mediano Plazo S/", "Inst. Deuda Mediano Plazo $",
                         "Inst. Deuda Corto Plazo S/", "Inst. Deuda Corto Plazo $", "Inst Deuda Duración Flexibl S/" , 
                         "Inst Deuda Duración Flexibl $" , "Instrumento de Deuda"]

categorias_renta_mixta = ["Renta Mixta Crecimiento Soles", "Renta Mixta Balanceado Dólares", "Renta Mixta Moderado Soles", 
                          "Renta Mixta Moderado Dólares"]

# Iterate over the dataframe's rows, 
for i in range(len(fondos_mutuos_dataframe_3)):
    if fondos_mutuos_dataframe_3[i]["Tipo Fondo"] in categorias_renta_fija:  # If the row's value for Tipo Fondo column is in categorias_renta_fija, add "Renta Fija" as the value for the row's Categoria column
        fondos_mutuos_dataframe_3[i]["Categoria"] = "Renta Fija"
    elif fondos_mutuos_dataframe_3[i]["Tipo Fondo"] in categorias_renta_mixta: # Else if the value is in categorias renta mixta, add "Renta Mixta" as the value for the row's categoria colun
        fondos_mutuos_dataframe_3[i]["Categoria"] = "Renta Mixta"
    elif fondos_mutuos_dataframe_3[i]["Tipo Fondo"] == "Flexible" or fondos_mutuos_dataframe_3[i]["Tipo Fondo"] == "Flexible Dólares":
        fondos_mutuos_dataframe_3[i]["Categoria"] = "Flexible"
    else:
        fondos_mutuos_dataframe_3.loc["Categoria"] = fondos_mutuos_dataframe_3.loc[i]["Tipo Fondo"] # else, set the value for the Categoria column = to the value in the "Tipo Fondo" column



print(fondos_mutuos_dataframe_3.head())

### Implementacion 2: using .loc

In [ ]:
import pandas as pd

fondos_mutuos_dataframe_3 = pd.read_csv("fondos-mutuos-table-3.csv")

# Get set of unique type of funds from the "Tipo Fondo" column
tipo_de_fondos = set(fondos_mutuos_dataframe_3["Tipo Fondo"])

categorias_renta_fija = ["Corto Plazo Soles","Corto Plazo Dólares", "Inst. Deuda Mediano Plazo S/", "Inst. Deuda Mediano Plazo $",
                         "Inst. Deuda Corto Plazo S/", "Inst. Deuda Corto Plazo $", "Inst Deuda Duración Flexibl S/" , 
                         "Inst Deuda Duración Flexibl $" , "Instrumento de Deuda"]

categorias_renta_mixta = ["Renta Mixta Crecimiento Soles", "Renta Mixta Balanceado Dólares", "Renta Mixta Moderado Soles", 
                          "Renta Mixta Moderado Dólares"]

# Alternative implementation using panad's .loc method
for i in range(len(fondos_mutuos_dataframe_3)):
    if fondos_mutuos_dataframe_3.loc[i, "Tipo Fondo"] in categorias_renta_fija:
        fondos_mutuos_dataframe_3.loc[i, "Categoria"] = "Renta Fija"
    elif fondos_mutuos_dataframe_3.loc[i, "Tipo Fondo"] in categorias_renta_mixta:
        fondos_mutuos_dataframe_3.loc[i, "Categoria"] = "Renta Mixta"
    elif fondos_mutuos_dataframe_3.loc[i, "Tipo Fondo"] == "Flexible" or fondos_mutuos_dataframe_3.loc[i, "Tipo Fondo"] == "Flexible Dólares":
        fondos_mutuos_dataframe_3.loc[i, "Categoria"] = "Flexible"
    else:
        fondos_mutuos_dataframe_3.loc[i, "Categoria"] = fondos_mutuos_dataframe_3.loc[i]["Tipo Fondo"]

#print(fondos_mutuos_dataframe_3.head())

for i in range(10):
    print(fondos_mutuos_dataframe_3.iloc[i])

## Task 7: max and min returns from fund dictionary

In [ ]:
fund = {
        "Tipo Fondo":"Renta Variable",
        "Fondo Mutuo":"CREDICORP ACCIONES GLOBAL",
        "Administradora":"CREDIFONDO SAF",
        "Fec. Inicio Operaci\u00f3n":"25\/03\/2013",
        "Moneda Cuota":"$",
        "Valor Cuota":180.92,
        "Rentabilidad 2024":0.0967,
        "Rentabilidad 2023":0.1656,
        "Rentabilidad 2022":-0.2041,
        "Rentabilidad 2021":0.1397,
        "Rentabilidad 2020":0.1303,
        "Rentabilidad 2019":0.2313,
        "Rentabilidad 2018":-0.1154,
        "Rentabilidad 2017":0.2009,
        "Rentabilidad 2016":0.0315,
        "Rentabilidad 2015":-0.0556,
        "Rentabilidad 2014":-0.0042,
        "Standard Dev":0.1384303816,
        "Return Lowest":-0.2041,
        "Return Highest":0.2313,
        "Avg Return Arithmetic":0.0561,
        "Patrimonio S\/.":23652842.5100000016,
        "Part\u00edcipes N":894,
        "A\u00f1os":11,
        "Categoria":"Renta Variable"
    }


# Maximum Return Subroutine

# Implementation 1:
fund_returns = []

for key in fund:
    if "Rentabilidad" in key:
        fund_returns.append(fund[key])

max_return = max(fund_returns)
print(max_return)

print("")


# Implementation 2"
fund_keys = list(fund.keys())
fund_returns2 = []

for key in fund_keys:
    if "Rentabilidad" in key:
        fund_returns2.append(fund[key])
max_return = max(fund_returns)


### Testing Highest Return Funciton

In [ ]:
import json
import os
print("Current working directory:", os.getcwd())   # note: this script runs 


file = open('fondos-mutuos-data-2.json') 

# Load json fie to a python list object
fondos_mutuos_list = json.load(file)  

def highest_return(fund):
    yearly_returns = []

    for key in fund:
        if "Rentabilidad" in key and fund[key] is not None:
            yearly_returns.append(fund[key])

    if len(yearly_returns) > 0:
        max_return = max(yearly_returns)
        fund["Highest Return"] = max_return
    
    else:
         fund["Highest Return"] = None  

    return fund 

for fund in fondos_mutuos_list:
    highest_return(fund)

for i in range(3):
    print(fondos_mutuos_list[i])


### Testing Min Return Function

In [ ]:
import json
import os

# Open json file with data 
file = open('fondos-mutuos-data-2.json') 

# Load json fie to a python list object
fondos_mutuos_list = json.load(file)  

def lowest_return(fund):
    yearly_returns = []

    for key in fund:
        if "Rentabilidad" in key and fund[key] is not None:
            yearly_returns.append(fund[key])

    if len(yearly_returns) > 0:  
        lowest_return = min(yearly_returns)
        fund["Lowest Return"] = lowest_return
    
    else:
        fund["Lowest Return"] = None 

    return fund

# Test Function:
for fund in fondos_mutuos_list:
    lowest_return(fund)

# Print first 3 funds:
for i in range(3):
    print(fondos_mutuos_list[i])



### Testing Arithmetic Average Return Function

In [ ]:
import json

# Open json file with data 
file = open('fondos-mutuos-data-2.json') 

# Load json fie to a python list object
fondos_mutuos_list = json.load(file)  

def arithmetic_mean(fund):

    fund_years = 0 

    sum_returns = 0

    for key in fund:
        if "Rentabilidad" in key and fund[key] is not None:
            sum_returns += fund[key] 

            if "2024" in key:
                fund_years += 0.5
            else: 
                fund_years += 1

    if fund_years > 0:
        arithmetic_mean = sum_returns / fund_years
        fund["Avg Return (Arithmetic)"] = arithmetic_mean

    else:
        fund["Avg Return (Arithmetic)"] = None

    return fund


# Test Function:
for fund in fondos_mutuos_list:
    arithmetic_mean(fund)

# Print first 3 funds:
for i in range(3):
    print(fondos_mutuos_list[i])

### Testing Standard Dev Function

In [ ]:
import json
import math

# Open json file with data 
file = open('fondos-mutuos-data-2.json') 

# Load json fie to a python list object
fondos_mutuos_list = json.load(file)  

def standard_dev(fund):

    mean = fund["Avg Return Arithmetic"]

    fund_years = 0 

    sum_diff_squares = 0

    for key in fund:
        if "Rentabilidad" in key and fund[key] is not None:
            fund_years += 1

            squared_diff = (fund[key] - mean)**2

            sum_diff_squares += (squared_diff)
    
    if fund_years > 1:
        stdev = math.sqrt(sum_diff_squares/fund_years)
        fund["Standard Deviation of Returns"] = stdev
    
    else:
        fund["Standard Deviation of Returns"] = None


# Test Function:
for fund in fondos_mutuos_list:
    standard_dev(fund)

# Print first 3 funds:
for i in range(3):
    print(fondos_mutuos_list[i])


